<a id="top"></a>
# AntakIA tutorial
***
### Using AntakIA with the GUI!

AntakIA helps you understand and explain your _black-box_ machine-learning models, by identifying the most relevant way of segregating your dataset and the best surrogate models to apply on these freshly created regions. In this notebook, we will show you how to use the automatic dyadic-clustering algorithm of AntakIA.

> This notebook is a tutorial on how to use AntakIA with the GUI. If you want to use the GUI, please refer to the [AntakIA without GUI tutorial](antakia_CH_no_gui.ipynb).
> 
> For more information about AntakIA, please refer to the [AntakIA documentation](https://ai-vidence.github.io/antakia/) or go to [AI-vidence's website](https://ai-vidence.com/).

__In this notebook, you will learn how to:__
- Create a dataset object from a CSV file
- Instanciate an AntakIA object
- Run the GUI to explore the dataset, the model, define regions and apply sub-models
- Visualize the results

## Context :

__Let's pretend that we are a real estate agent and that we want to predict the price of a house based on its characteristics.__ We have a dataset of more than 20000 blocks of houses, each block being described by 8 features (e.g. medium income of the owners, number of rooms, etc.). We also have the price of each block of houses. We already trained a machine-learning model (in our case, a simple XGBoost) that will predict the price of a house based on its characteristics. This is very helpful to estimate the price of a house that we want to sell !

__The main issue is the following :__ we want to explain to our customers why their house is worth a certain price. We can't just show them the machine-learning model, because it is a _black-box_ model. We need to find a way to explain the price of a house based on its characteristics. This is where AntakIA comes in handy !

We start by importing the necessary libraries.

In [1]:
import pandas as pd 

Then, our dataset. Ours is [this one](https://inria.github.io/scikit-learn-mooc/python_scripts/datasets_california_housing.html); it can be found in the `data` folder of this repository.

In [2]:
df = pd.read_csv('../data/california_housing.csv').drop(['Unnamed: 0'], axis=1)

After cleaning a bit our data, we want to specifically focus on __San Francisco__ and its surroundings.

In [3]:
# Remove outliers:
df = df.loc[df['Population']<10000] 
df = df.loc[df['AveOccup']<6]
df = df.loc[df['AveBedrms']<1.5]
df = df.loc[df['HouseAge']<50]

# Only San Francisco :
df = df.loc[(df['Latitude']<38.07)&(df['Latitude']>37.2)]
df = df.loc[(df['Longitude']>-122.5)&(df['Longitude']<-121.75)]

Note that we already computed some explanatory values (in our case, SHAP values) and saved them in the CSV file. This is not necessary, as AntakIA can do it, but it will save us some computation time!

In [4]:
X = df.iloc[:,0:8] # the dataset
Y = df.iloc[:,9] # the target variable
shapValues = df.iloc[:,[10,11,12,13,14,15,16,17]] # the SHAP values

We also have a trained XGBoost model that we will use to predict the price of a house.

In [5]:
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor(random_state = 9)
model.fit(X, Y)
print('model fitted')

model fitted


__Let's now import `antakia`!__

In [6]:
from antakia.data import Dataset, ExplanationMethod, ExplanationDataset

from antakia.antakia import AntakIA
dataset = Dataset(X, y=Y)
shapExplanations = ExplanationDataset(shapValues, ExplanationMethod.SHAP)
atk = AntakIA(dataset,model, shapExplanations)

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

/Users/laurent/Documents/antakia/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/laurent/Documents/antakia/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/laurent/Documents/antakia/lib/python3.10/site-packages/umap/distanc

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [7]:
atk = AntakIA(dataset,model, shapExplanations)

In [8]:
atk.startGUI()

Output()

AttributeError: 'GUI' object has no attribute '_leftVSFigure'